Silver Layer Transformation Script

Data Acess Via Azure App 

Data Loading From Bronze

In [0]:
# Access data from the Data Lake
df_Calendar = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Calendar")

In [0]:
# Access data from the Data Lake
df_Customers = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Customers")

In [0]:
# Access data from the Data Lake
df_Product_Categories = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Product_Categories")

In [0]:
# Access data from the Data Lake
df_Products = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Products")

In [0]:
# Access data from the Data Lake
df_Returns = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Returns")

In [0]:
# Access data from the Data Lake
df_Product_Subcategories = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Product_Subcategories")

In [0]:
# Access data from the Data Lake
df_Territories = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Territories")

In [0]:
# Access data from the Data Lake
df_Sales_2015 = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Sales_2015")

In [0]:
# Access data from the Data Lake
df_Sales_2016 = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Sales_2016")

In [0]:
# Access data from the Data Lake
df_Sales_2017 = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@mabeddatalake.dfs.core.windows.net/AdventureWorks_Sales_2017")

Transformation

In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [0]:
df_Calendar.display()

In [0]:
df_Calendar = df_Calendar.withColumn('Month',month(col('Date')))\
            .withColumn('Year',year(col('Date')))

df_Calendar.display()

In [0]:
df_Calendar.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@mabeddatalake.dfs.core.windows.net/AdventureWorks_Calendar")\
            .save()

In [0]:
df_Customers.display()

In [0]:
df_Customers = df_Customers.withColumn('fullName',concat_ws(' ',col('Prefix'),col('FirstName'),col('lastName')))

In [0]:
df_Customers.display()

In [0]:
df_Customers.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@mabeddatalake.dfs.core.windows.net/AdventureWorks_Customers")\
            .save()

In [0]:
df_Product_Subcategories.display()

In [0]:
df_Product_Subcategories.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@mabeddatalake.dfs.core.windows.net/AdventureWorks_Product_Subcategories")\
            .save()


In [0]:
df_Products.display()

In [0]:
df_Products = df_Products.withColumn('ProductSKU',split(col('ProductSKU'),'-')[0])\
                .withColumn('ProductName',split(col('ProductName'),' ')[0])

In [0]:
df_Products.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@mabeddatalake.dfs.core.windows.net/AdventureWorks_Products")\
            .save()


In [0]:
df_Returns.display()

In [0]:
df_Returns.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@mabeddatalake.dfs.core.windows.net/AdventureWorks_Returns")\
            .save()


In [0]:
df_Territories.display()

In [0]:
df_Territories.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@mabeddatalake.dfs.core.windows.net/AdventureWorks_Territories")\
            .save()


In [0]:
# Union all DataFrames
df_Sales = df_Sales_2015.union(df_Sales_2016).union(df_Sales_2017)

# Optional: remove duplicates if needed
df_Sales = df_Sales.dropDuplicates()

# Show combined DataFrame
df_Sales.display()


In [0]:
df_Sales = df_Sales.withColumn('StockDate',to_timestamp('StockDate'))

In [0]:
df_Sales = df_Sales.withColumn('OrderNumber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_Sales = df_Sales.withColumn('multiply',col('OrderLineItem')*col('OrderQuantity'))
df_Sales.display()

In [0]:
df_Sales.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@mabeddatalake.dfs.core.windows.net/AdventureWorks_Sales")\
            .save()

In [0]:
df_Sales.groupBy('OrderDate').agg(count('OrderNumber').alias('total_order')).display()